# Resistive Wall Wakefield

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pmd_beamphysics.wakefields.resistive_wall import (
    ResistiveWallWakefield,
    epsilon_0,
)

from pmd_beamphysics import ParticleGroup
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
radius_ref = 2.5e-3

wake = ResistiveWallWakefield.from_material(
    material="copper-slac-pub-10707", radius=radius_ref, geometry="round"
)

wake.plot()

In [ ]:
print(wake.to_bmad())

# Compare with SLAC-PUB-10707

In [ ]:
raw_data = np.loadtxt("../data/SLAC-PUB-10707-digitized-Fig4-AC-Cu.csv", delimiter=",")

# Convert to SI
zref, Wref = (
    raw_data[:, 0] * 1e-6,
    raw_data[:, 1] * 4 / radius_ref**2 / (4 * np.pi * epsilon_0),
)
plt.plot(zref, Wref)

In [ ]:
zlist = np.linspace(0, 300e-6, 200)
Wz = wake.pseudomode(-zlist)

fig, ax = plt.subplots()
ax.plot(zlist * 1e6, Wz * 1e-12, label=f"ResistiveWallWakefield {wake.geometry}")

ax.plot(zref * 1e6, Wref * 1e-12, "--", label="Fig. 4 AC-Cu from SLAC-PUB-10707 2004")
plt.legend()

ax.set_xlabel(r"$-z$ (µm)")
ax.set_ylabel(r"$W_z$ (V/pC/m)")

# Test with Bmad

In [ ]:
LAT = f"""
no_digested
parameter[lattice] = LCLS
parameter[e_tot] = 14e9 
parameter[particle] = electron        
parameter[geometry] = open

beginning[beta_a] = 10
beginning[beta_b] = 10


p1: pipe, L = 1

lat: line = (p1)

*[sr_wake] = {wake.to_bmad()}

use, lat

"""

print(LAT)

In [ ]:
from pytao import Tao

In [ ]:
tao = Tao.from_lattice_contents(LAT, noplot=True)

In [ ]:
beam_init = dict(
    n_particle=100_000,
    a_norm_emit=1e-12,
    b_norm_emit=1e-12,
    sig_z=20e-6,
    sig_pz=1e-9,
    n_bunch=1,
    bunch_charge=1e-9,
    random_engine="quasi",
)

cmds = []
for k, v in beam_init.items():
    cmd = f"set beam_init {k} = {v}"
    cmds.append(cmd)
    print(cmd)

tao.cmds(cmds, suppress_lattice_calc=False)

In [ ]:
# Peak current
beam_init["bunch_charge"] / (np.sqrt(2 * np.pi) * beam_init["sig_z"]) * 299792458

In [ ]:
%%time
tao.track_beam()

In [ ]:
P0 = ParticleGroup(data=tao.bunch_data("beginning"))
P1 = ParticleGroup(data=tao.bunch_data("end"))
P0.drift_to_t()
P1.drift_to_t()
P1.plot(
    "delta_z/c",
    "energy",
    bins=200,
    ylim=(14e9 - 200e3, 14e9 + 100e3),
    xlim=(-300e-15, 300e-15),
    nice=False,
)

In [ ]:
e1 = P1["mean_energy"]
e0 = P0["mean_energy"]
(e1 - e0)

In [ ]:
def gaussian(z, sigma=beam_init["sig_z"], mean=0):
    return 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-0.5 * ((z - mean) / sigma) ** 2)


z_min, z_max, dz = -200e-6, 200e-6, 0.1e-6
z = np.arange(z_min, z_max + dz, dz)

density = gaussian(z)
charge_density = density * beam_init["bunch_charge"]

In [ ]:
iwake = -wake.convolve_density(charge_density, dz=dz)
plt.plot(iwake)

In [ ]:
np.sum(density * iwake) * dz  # eV/m

In [ ]:
# Get overall length for comparison per length
L_tot = tao.ele_head("end")["s"] - tao.ele_head("beginning")["s"]
L_tot

In [ ]:
z_particles = P1["delta_z"]
de_particles = P1.energy - P0["mean_energy"]

zscale = 1 / 1e-6
fig, ax = plt.subplots()
ax.scatter(
    z_particles * zscale,
    de_particles / L_tot,
    label="tracking",
    marker=".",
    color="black",
)

# ax.plot(z_conv * zscale, conv * N * e, color='red', label='convolution')

ax.plot(z * zscale, iwake, color="red", label="convolution")

ax.legend()
ax.set_xlim(z_particles.min() * zscale, z_particles.max() * zscale)
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W_z$ (eV/m)")

# Detailed wake from tracking


In [ ]:
z = P0.z
weight = P0.weight

In [ ]:
%%time
zwake = wake.pseudomode.particle_kicks(z, weight)
len(zwake)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(z * zscale, zwake, label="Python", color="black")
ax.scatter(
    z_particles * zscale,
    de_particles / L_tot,
    label="Bmad tracking",
    marker=".",
    color="blue",
)
ax.legend()
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W_z$ (eV/m)")